# Cədvəlin verilən sıra nömrələri əsasında bərabər aralıqlı hissələrə bölünməsi

Bu "notebook"u yaratmaqda məqsədim öz qarşıma çıxan bir problemi həll etmək idi. Əlimdə içində yüz minlərlə sətir olan bir cədvəl var idi. Mən həmin cədvəli ardıcıl identifikasiya nömrələri ilə hər biri 1000-lik aralığı əhatə edən kiçik cədvəllərə çevirməli idim. Məsələn, 1001-dən 2000-ə qədər İD-ləri olan bütün sətirlər bir cədvələ, 2001-3000 aralığındakı nömrələr digər cədvələ yazılmalı idi. Problem həm də ondan ibarət idi ki, İD-lər ardıcıl gəlmirdi. Məsələn 3015-dən sonrakı İD 3020 ola bilərdi, arada bir sıra İD-lər isə cədvəldə yox idi. Buna görə də 1000-lik aralıqlı cədvəllərdə say bərabər olmayacaqdı. Yəni cədvəli İD-yə görə sıralayıb, sonra 1000-1000 bölə bilməzdim. Bu zaman cədvəllər mənim istədiyim kimi 3001-4000 kimi olmazdı, arada istifadə olunamayan İD-lərə görə, 1000 ədəd götürsəm axırıncı ədəd 4000-dən böyük olardı və siyahə səliqəli bölünməzdi. Ona görə aşağıdakı kimi həll yolu fikirləşdim. 

Bəlkə də, bu bir o qədər çox rast gəlinən problem deyil, yaxud da çox sadədir. Amma düşündüm ki məndən başqa da kiminsə işinə yaraya bilər. Odur ki, bölüşmək qərarına gəldim.

İlk olaraq lazımi paketləri yükləyirik. Edəcəyimiz iş sadəcə əlimizdəki cədvəli hissələrə bölmək olacağı üçün sadəcə "Pandas" paketi kifayətdir. "Numpy" paketini isə "list" formatındakı ddəyişənləri "print()" edərkən ekranda daha yığcam çap olunmasını təmin etmək üçün istifadə edəcəyik.

In [3]:
import pandas as pd
import numpy as np

Hissələrə bölmək istədiyimiz cədvəli "notebook"a yükləməliyik. Bunun üçün faylın yerləşdiyi "directory"yə, faylın adına və "pd.read_excel" funksiyasına ehtiyacımız var.

In [4]:
# read_path = 'C:\\Users\\...\\'
# read_file_name = "bina_az_extracted_2007000_2023000.xlsx"
# bina_az_extr_df = pd.read_excel(read_path+read_file_name)
# bina_az_extr_df.head()

Hazırda isə biz "notebook"un hər kəs üçün rahat icra edilə bilməsindən ötrü özümüz "Pandas"ın köməyilə cədvəl yaradayağıq. Mən ID üçün aralığı 0 - 10.000 arası götürdüm. Siz bu aralığı dəyişə bilərsiniz.

In [19]:
# 1) random ids grouped by 1000 (or any other value you want)
step_size = 1000
first = 0
last = first + step_size
num_of_elements = 1000
iter = 0
id_list = []
while last <= 11_000:
  id_list.extend(np.random.randint(first,last,num_of_elements))
  # print(iter)
  # print(last)
  iter = iter + 1
  first = last
  last = last + step_size

# 2) remove duplicates
id_series = pd.Series(id_list).drop_duplicates()

# 3) add random characteristics

# is employed
is_emp_series = np.random.choice(['yes', 'no'],len(id_series), [0.4, 0.6]) 

# education level
educat_series  = np.random.choice(['1st', '2nd', '3rd', '4th'],len(id_series), [0.1, 0.3, 0.4, 0.2]) 

# salary satisfaction
sal_sat_series = np.random.choice(['1', '2', '3', '4', '5'],len(id_series), [0.1, 0.3, 0.3, 0.2, 0.1])


# 4) build a DataFrame
random_df = pd.DataFrame({'ID': id_series,
                          'is_employed': is_emp_series,
                          'education_level': educat_series,
                          'salary_satisfaction': sal_sat_series}) 

# 5) shuffle it
random_df = random_df.sample(frac = 1).reset_index(drop = True)

# result
random_df.head()

,ID,is_employed,education_level,salary_satisfaction
0,1167,no,3rd,2
1,656,no,2nd,2
2,4047,no,4th,5
3,5133,yes,3rd,3
4,5701,no,4th,1


Sonra cədvəlimizi bölmək üçün aralıq seçirik (size = 1000). Mən burada İD nömrələrinin 1000-lik aralıqlara bölərək cədvəllər yaratmaq istəyirəm. Ona görə də "size" dəyişənini 1000 olaraq mənimsədirəm. Və 0-dan ən cədvəldəki ən böyük İD-dən böyük olan ən kiçik 1000-ə bölünən ədəd arasında 1000 vahid artan ədədi silsilə qururam. (Məsələn siyahıda axırıncı İD 8502-dirsə, ondan böyük ən kiçik 1000-ə bölünən ədəd 9000-dir. Ona görə siyahı belə olacaq: [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]

Daha sonra siyahını yaradırıq.

In [12]:
# define the size for split
size = 1000
# create a list of multiples of 1000 (in this case)
# between 0 and the first multiple of 1000 that is greater than last ID number
# in original list
id_list_orig = list(random_df['ID'])

# find the minimum multiple of 1000 that is greater than maximum ID number
end_id = (max(id_list_orig)//size+1)*size + 1

# creating list of IDs
list_threshs = [*range(0,end_id,size)]
list_threshs

# we will use this list in both methods

[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000]


Daha sonra 2 üsuldan biri ilə dataframeləri hissələrə bölə bilərik.

1-ci üsulda siyahıdakı ədədlərə əsasən İD aralıqlarını göstərən sütun yaradırıq. 

In [13]:
# method 1 - Creating a column for ID category
# create a column for ID group as the multiples of 1000
random_df['ID_Group'] = \
                        [ids if (ids%size == 0) & (ids != 0)
                            else (ids//size + 1) * size
                        
                            for ids in random_df['ID']] 
random_df.head()

,ID,is_employed,education_level,salary_satisfaction,ID_Group
0,2889,yes,2nd,1,3000
1,681,yes,3rd,3,1000
2,6478,no,2nd,5,7000
3,8880,yes,2nd,1,9000
4,2176,yes,3rd,2,3000


Sonra isə qrup nömrəsi eyni olan sətirləri bir dataframe-də qruplaşdırırıq. Dataframeləri loop-un daxilində yaratmaq üçün "globals()" funksiyasından istifadə edirik. Bu funksiya bizə daxil etdiyimiz "string" yəni mətni dəyişən adına çevirməkdə kömək edir.

In [14]:
df_list = []
# for the multiples of 1000 in the list except 0
for i in list_threshs[1:]:
  # create a sliced dataframe where 'ID_Group' is equal to that multiple of 1000
  globals()['df_'+str(i)] = random_df[random_df['ID_Group'] == i]

  # append the name of df to a list for further reference
  df_list.append('df_'+str(i))
print(df_list)

['df_1000', 'df_2000', 'df_3000', 'df_4000', 'df_5000', 'df_6000', 'df_7000', 'df_8000', 'df_9000', 'df_10000', 'df_11000']


2-ci üsulda "pd.cut()" metodundan istifadə edirik. Burada "pd.cut()" vasitəsilə cədvələ İD sütunu əsasında interval sütunu yaradırıq. 

In [15]:
# method 2 - Cut intervals of 'ID' column
# based on the list created in 1st step
random_df['ID_Bins'] = pd.cut(random_df['ID'], bins = list_threshs)
random_df.head()

,ID,is_employed,education_level,salary_satisfaction,ID_Group,ID_Bins
0,2889,yes,2nd,1,3000,"(2000, 3000]"
1,681,yes,3rd,3,1000,"(0, 1000]"
2,6478,no,2nd,5,7000,"(6000, 7000]"
3,8880,yes,2nd,1,9000,"(8000, 9000]"
4,2176,yes,3rd,2,3000,"(2000, 3000]"


Sonra yuxarıda yaratdığımız "list_threshs" listi ilə sort olunmuş təkrarlanmayan ("unique") interval siyahısı əsasında bir "dictionary" yaradırıq. Burada "list_threshs" listi bizə yeni yaranan dataframeləri loop-un daxilində adlandırmağa kömək edəcək.

Sonra "dictionary" daxilindəki "key" və "value" cütündən istifadə edərək loop formalaşdırırıq və onun daxilində hər aralıq üçün eyni aralığa uyğun gələn sətirləri özündə birləşdirən dataframelər yaradırıq.

In [16]:
# create a dictionary
# from multiples of 1000 in 'list_threshs' as keys
# and 'ID_Bins' as values
bin_list =  sorted(random_df["ID_Bins"].value_counts().index.to_list())
bin_dict = {list_threshs[1:][i]: bin_list[i] for i in range(len(bin_list))}


print('Bin Dictionary')
print(bin_dict)
print('='*100)


df_list = []
# for multiples of 1000 in 'list_threshs' as keys
# and 'ID_Bins' as values
for key , value in bin_dict.items():
  # create a sliced dataframe where 'ID_Bins' = given interval
  # and name the dataframe as the given multiple of 1000
  globals()['df_'+str(key)] = random_df[random_df['ID_Bins'] == value]
  df_list.append('df_'+str(key))


print('')
print('List of Created DataFrames')
print(df_list)
print('='*100)

Bin Dictionary
{1000: Interval(0, 1000, closed='right'), 2000: Interval(1000, 2000, closed='right'), 3000: Interval(2000, 3000, closed='right'), 4000: Interval(3000, 4000, closed='right'), 5000: Interval(4000, 5000, closed='right'), 6000: Interval(5000, 6000, closed='right'), 7000: Interval(6000, 7000, closed='right'), 8000: Interval(7000, 8000, closed='right'), 9000: Interval(8000, 9000, closed='right'), 10000: Interval(9000, 10000, closed='right'), 11000: Interval(10000, 11000, closed='right')}

List of Created DataFrames
['df_1000', 'df_2000', 'df_3000', 'df_4000', 'df_5000', 'df_6000', 'df_7000', 'df_8000', 'df_9000', 'df_10000', 'df_11000']


Cədvəlləri orijinala uyğunlaşdırmaq üçün yaratdığımız yeni sütunları axırda bütün cədvəllərdən silirik.

In [17]:
# print summary results
for df_name in df_list:
  df = globals()[df_name]
  print('DataFrame name:'       , df_name)
  print('Min ID value:'         , df['ID'].min())
  print('Max ID value:'         , df['ID'].max())
  print('Interval:'             , df['ID_Bins'].unique())
  print('Group:'                , df['ID_Group'].unique())
  print('='*100)
  print('')
  # and for each sliced dataframe drop the unncessary columns
  # (it is an optional step)
  globals()[df_name] = globals()[df_name].drop(['ID_Bins', 'ID_Group'], axis = 1)

DataFrame name: df_1000
Min ID value: 1
Max ID value: 1000
Interval: [(0, 1000]]
Categories (11, interval[int64, right]): [(0, 1000] < (1000, 2000] < (2000, 3000] < (3000, 4000] <
                                          ... < (7000, 8000] < (8000, 9000] <
                                          (9000, 10000] < (10000, 11000]]
Group: [1000]

DataFrame name: df_2000
Min ID value: 1001
Max ID value: 1999
Interval: [(1000, 2000]]
Categories (11, interval[int64, right]): [(0, 1000] < (1000, 2000] < (2000, 3000] < (3000, 4000] <
                                          ... < (7000, 8000] < (8000, 9000] <
                                          (9000, 10000] < (10000, 11000]]
Group: [2000]

DataFrame name: df_3000
Min ID value: 2001
Max ID value: 2998
Interval: [(2000, 3000]]
Categories (11, interval[int64, right]): [(0, 1000] < (1000, 2000] < (2000, 3000] < (3000, 4000] <
                                          ... < (7000, 8000] < (8000, 9000] <
                                    

Sonda yaratdığımız xüsusi "write_dfs" funksiyası ilə içində cədvəllərin adları olan siyahıdakı bütün cədvəlləri fayla çevirib qeyd etdiyimiz "directory"də yadda saxlayırıq.

In [18]:
# write_path = path = 'C:\\Users\\...\\'
# to_file = ".csv"
# file_name_pref = "bina_az_ext_" 

# create 'write_dfs' function 
# to write dataframes as file ('.csv,' '.xlsx', '.pkl', etc.)
def write_dfs(df_list, write_path, file_name_pref, encode, to_file = '.csv'):
  '''
  df_list - list of given names of dataframes to convert to file;

  write_path - the folder directory where we want to save files;

  file_name_pref - the file name prefix we want to use 
                   as the same pattern for all the created files;

  encode - used for recognition to characters in different languages
           while writing dataframe to a file.
           For Azerbaijani use: 'utf-8-sig';

  to_file - the file format that we want to create 
            ('.csv,' '.xlsx', '.pkl', etc.)
  '''
  for df in df_list:
      start = globals()[df][0]
      end = globals()[df][-1]
      globals()[df].to_csv(path + 
                          file_name_beg +
                          str(start) +
                          "_" +
                          str(end) +
                          file_type, encoding = encode)

Siz də "notebook"u endirib funksiyaları öz cədvəllərinizə tətbiq edə bilərsiniz. Ümid edirəm ki, hazırladığım kodu izah edə bildim. Yaranan hər hansısa bir sual olarsa, əlaqə saxlamaqdan çəkinməyin:

https://www.challengersdeep.com/